In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("..//Data/compas-scores-two-years.csv")
#display(df.columns)

In [3]:
#if we want to compare and get same answers as Compas Analysis.ipynb DO NOT run this cell (will remove rows)
df = df[df["days_b_screening_arrest"] <= 30]
df = df[df["days_b_screening_arrest"] >= -30]

### Oppgave 1b

Omgjøre score_text til to kategorier, low --> 0, medium og high --> 1

In [4]:
#jobber med mindre dataframe
sdf = df[['sex', 'race', 'two_year_recid', 'score_text']]
sdf

,sex,race,two_year_recid,score_text
0,Male,Other,0,Low
1,Male,African-American,1,Low
2,Male,African-American,1,Low
5,Male,Other,0,Low
6,Male,Caucasian,1,Medium
...,...,...,...,...
7209,Male,African-American,0,Medium
7210,Male,African-American,0,Low
7211,Male,Other,0,Low
7212,Female,African-American,0,Low


In [5]:
#code score_text into high_risk
sdf['high_risk'] = sdf['score_text'].apply(lambda score: 0 if score == 'Low' else 1) 

sdf['all'] = sdf['score_text'].apply(lambda x: 'All') #need this for groupby object later
sdf

<ipython-input-5-ada2cbb63ede>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdf['high_risk'] = sdf['score_text'].apply(lambda score: 0 if score == 'Low' else 1)
<ipython-input-5-ada2cbb63ede>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdf['all'] = sdf['score_text'].apply(lambda x: 'All') #need this for groupby object later


,sex,race,two_year_recid,score_text,high_risk,all
0,Male,Other,0,Low,0,All
1,Male,African-American,1,Low,0,All
2,Male,African-American,1,Low,0,All
5,Male,Other,0,Low,0,All
6,Male,Caucasian,1,Medium,1,All
...,...,...,...,...,...,...
7209,Male,African-American,0,Medium,1,All
7210,Male,African-American,0,Low,0,All
7211,Male,Other,0,Low,0,All
7212,Female,African-American,0,Low,0,All


In [6]:
def crosstab(df):
    return pd.crosstab(df.high_risk, df.two_year_recid, margins=True)

In [7]:
def false_positive(cross_table, label):
    # FP / (FP + TN)
    return cross_table.loc[(label, 1), 0]/ cross_table.loc[(label, 'All'), 0]

In [8]:
def true_positive(cross_table, label):
    # TP / (TP + FN)
    return cross_table.loc[(label, 1), 1] / cross_table.loc[(label, 'All'), 1]

In [9]:
def true_negative(cross_table, label):
    # TN / (TN + FP)
    return cross_table.loc[(label, 0), 0] / cross_table.loc[(label, 'All'), 0]

In [10]:
def false_negative(cross_table, label):
    # FN / (FN + TP)
    return cross_table.loc[(label, 0), 1] / cross_table.loc[(label, 'All'), 1]

In [11]:
#correct?
#using bayes theorem - ish
def p_recid_g_high(cross_table, label):
    return cross_table.loc[(label, 1), 1] / cross_table.loc[(label, 1), 'All']

In [12]:
groupobj = [sdf.groupby('all'), sdf.groupby('sex'), sdf.groupby('race')]

In [13]:
cross_tables = [group.apply(crosstab) for group in groupobj]
cross_tables

[two_year_recid     0     1   All
 all high_risk                   
 All 0           2345  1076  3421
     1           1018  1733  2751
     All         3363  2809  6172,
 two_year_recid       0     1   All
 sex    high_risk                  
 Female 0           532   167   699
        1           230   246   476
        All         762   413  1175
 Male   0          1813   909  2722
        1           788  1487  2275
        All        2601  2396  4997,
 two_year_recid                 0     1   All
 race             high_risk                  
 African-American 0           873   473  1346
                  1           641  1188  1829
                  All        1514  1661  3175
 Asian            0            21     3    24
                  1             2     5     7
                  All          23     8    31
 Caucasian        0           999   408  1407
                  1           282   414   696
                  All        1281   822  2103
 Hispanic         0           258 

In [14]:
functions = [false_positive, false_negative, true_positive, true_negative, p_recid_g_high]
d = {}
for function in functions:
    inner_dict = {}
    for sort, table in zip(groupobj, cross_tables):
        for group in sort.groups:
            if not group in ['Asian', 'Hispanic', 'Native American', 'Other']:
                inner_dict[str(group)] = round(function(table, str(group)), 4)
    d[function.__name__] = inner_dict
d

{'false_positive': {'All': 0.3027,
  'Female': 0.3018,
  'Male': 0.303,
  'African-American': 0.4234,
  'Caucasian': 0.2201},
 'false_negative': {'All': 0.3831,
  'Female': 0.4044,
  'Male': 0.3794,
  'African-American': 0.2848,
  'Caucasian': 0.4964},
 'true_positive': {'All': 0.6169,
  'Female': 0.5956,
  'Male': 0.6206,
  'African-American': 0.7152,
  'Caucasian': 0.5036},
 'true_negative': {'All': 0.6973,
  'Female': 0.6982,
  'Male': 0.697,
  'African-American': 0.5766,
  'Caucasian': 0.7799},
 'p_recid_g_high': {'All': 0.63,
  'Female': 0.5168,
  'Male': 0.6536,
  'African-American': 0.6495,
  'Caucasian': 0.5948}}